# Imports

In [ ]:
import glob
import os
import pandas as pd
from typing import Dict, List, Union
import json

os.getcwd()

# Set Dir, Load Files

# Functions for detecting and correcting old JSONS, converting them to a CSV and back to JSON, and estimating protocol length from a group of files

In [ ]:
from typing import List

# Function to load JSON from a file path
def load_json(file_path):
    with open(file_path, 'r') as f:
        return json.load(f)


def estimate_gesture_length(file_paths: List[str]) -> Dict[str, List[float]]:
    """
    Estimate the length of each gesture in each JSON file.
    
    Parameters:
        file_paths (List[str]): List of paths to the JSON files.
    
    Returns:
        Dict[str, List[float]]: A dictionary where keys are file names and values are lists of 
                                 estimated lengths for each gesture in that file, in minutes.
    """
    
    # Initialize an empty dictionary to hold the estimated lengths
    estimated_lengths = {}

    # Loop through each file to estimate gesture lengths
    for file_path in file_paths:
        file_name = file_path.split('/')[-1]
        json_data = load_json(file_path)
        
        if 'gests' in json_data:
            file_lengths = []
            for gest in json_data['gests']:
                # Calculate the estimated length for the current gesture
                rep_count = gest.get('repCount', 0)
                duration = gest.get('duration', 0)
                
                total_duration_ms = rep_count * duration
                total_duration_minutes = (total_duration_ms / 1000) / 60
                
                file_lengths.append(total_duration_minutes)
            
            estimated_lengths[file_name] = file_lengths

    return estimated_lengths


def csv_to_json_files(input_csv_path: str, output_folder: str, version: float) -> List[str]:
    """
    Convert a CSV file back to individual JSON files, one for each test_number.
    
    Parameters:
        input_csv_path (str): Path to the input CSV file.
        output_folder (str): Folder where the output JSON files will be saved.
    
    Returns:
        List[str]: List of paths to the generated JSON files.
    """
    
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Load the CSV file into a DataFrame
    df = pd.read_csv(input_csv_path)
    
    if version == 0.54:
        df.rename(columns={
            'startStimulus': 'startInference',
            'endStimulus': 'endInference',
            'activeClass': 'name',
        }, inplace=True)
        
    
    # Initialize a list to hold the paths to the generated JSON files
    output_file_paths = []
    
    # Group the DataFrame by test_number
    grouped = df.groupby('test_number')
    
    for test_number, group in grouped:
        # Extract unique values for the columns used to construct the file name
        protocol_group = group['protocol_group'].iloc[0] if pd.notna(group['protocol_group'].iloc[0]) else ''
        arm_posture = group['arm_posture'].iloc[0] if pd.notna(group['arm_posture'].iloc[0]) else ''
        wrist_posture = group['wrist_posture'].iloc[0] if pd.notna(group['wrist_posture'].iloc[0]) else ''
        hand_posture = group['hand_posture'].iloc[0] if pd.notna(group['hand_posture'].iloc[0]) else ''
        body_activity = group['body_activity'].iloc[0] if pd.notna(group['body_activity'].iloc[0]) else ''
        handedness = group['handedness'].iloc[0] if pd.notna(group['handedness'].iloc[0]) else ''
        
        # Construct the file name
        file_name = f"{test_number}-{protocol_group}-{arm_posture}-{wrist_posture}-{hand_posture}-{body_activity}-{handedness}.json"
        
        # Create the JSON data structure
        json_data = {
            'name': file_name.split('.json')[0],  # Update the 'name' to match the file name
            'gests': group.drop(['test_number', 'protocol_group', 'arm_posture', 'file_name', 'wrist_posture', 'hand_posture', 'body_activity', 'handedness'], axis=1).to_dict('records')
        }
        
        # Save the JSON data to a file
        output_file_path = os.path.join(output_folder, file_name)
        with open(output_file_path, 'w') as f:
            json.dump(json_data, f, indent=4)
        
        output_file_paths.append(output_file_path)
    
    return output_file_paths

import shutil

def rename_files_with_double_digit_test_number(input_folder: str, output_folder: str) -> List[str]:
    """
    Rename files in a folder such that test numbers less than 10 are made double digits (e.g., 1 becomes 01).
    
    Parameters:
        input_folder (str): Folder containing the files to be renamed.
        output_folder (str): Folder where the renamed files will be saved.
    
    Returns:
        List[str]: List of paths to the renamed files.
    """
    
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Initialize a list to hold the paths to the renamed files
    renamed_file_paths = []
    
    # Loop through each file in the input folder
    for file_name in os.listdir(input_folder):
        if 'ipynb_checkpoints' not in file_name:
            # Extract the test number from the file name
            test_number = file_name.split('-')[0]

            # Convert single-digit test numbers to double digits
            if len(test_number) == 1:
                test_number = f"0{test_number}"
                
                """
        
        BUGGED WHEN TEST NUMBER EXISTS ELSEWHERE IN FILE NAME
        
        
        """

            # Construct the new file name
            new_file_name = file_name.replace(file_name.split('-')[0], test_number)

            # Copy the file to the output folder with the new name
            input_file_path = os.path.join(input_folder, file_name)
            output_file_path = os.path.join(output_folder, new_file_name)
            shutil.copy(input_file_path, output_file_path)

            renamed_file_paths.append(output_file_path)
    
    return renamed_file_paths

# make changes based on differences between old and new JSONS and correct protocol_ids to be incremental

# Convert the final CSV file back into JSON by test number

In [ ]:
# Test the function with 
output_folder = 'new_json'
output_file_paths = csv_to_json_files('defender.csv', output_folder, version = 0.54) # Enter corresponding csv file name in folder with script e.g. test.csv
output_file_paths[:5]  # Show first 5 output file paths for verification

# make sure files can be sorted incrementally by replacing any single digita number with double digit (eg. 1 --> 01)

In [ ]:
# Test the function with the existing output folder
renamed_output_folder = 'final_json'
renamed_file_paths = rename_files_with_double_digit_test_number(output_folder, renamed_output_folder)
renamed_file_paths[:5]  # Show first 5 renamed file paths for verification

In [ ]:
files = glob.glob(f'{renamed_output_folder}/*.json')
files

In [ ]:
# Test the function
estimated_lengths = estimate_gesture_length(files)
estimated_lengths

# Calculate the total estimated length for the entire set of files
total_length_minutes = sum(sum(lengths) for lengths in estimated_lengths.values())
print(f' {total_length_minutes} minutes')